In [1]:
# 2024 / 01 / 03
# Quad v2 dynamics

In [2]:
from eom import *
from sympy import symbols, factor
from sympy import simplify
from sympy.physics.mechanics import *
from sympy import sin, cos, symbols, Matrix, solve
from sympy.physics.vector import init_vprinting
import numpy as np
init_vprinting()

## System dynamics ##

The system to be controlled is an dual-moiton inverted pendulum on a cart (see next Figure).  

<img src="dual_motion.png" width="800" align="center"/>

Parameter description  
$m_t$ = 몸체 무게  
$m_c$ = 슬라이더 무게  
$m_w$ = 휠 무게  
$r$ = 휠 반지름  
$\theta_p $ = 몸체의 기울기(그림에서 $\theta$)  
$\theta_w $ = 휠 각도  
$x_l$ = 슬라이더 위치  
$h_t$ = 휠 중심부터 몸체 중심 까지의 거리  
$h_c$ = 휠 중심부터 슬라이더 중심 까지의 거리  
$I_t$ = 몸체 관성 모멘트  
$I_c$ = 슬라이더 관성 모멘트  
$I_w$ = 휠 관성 모멘트  

Lagrangian dynamics system parameter = $[\theta_w\; \theta_p\; x_l]$



In [3]:
# Define Symbolic Variables
x, x_l, theta = dynamicsymbols('x, x_l, theta')
phi = dynamicsymbols('phi')
F, F_l = dynamicsymbols('F, F_l')
r, h_c, h_t = symbols('r, h_c, h_t')
I_w, I_c, I_t = symbols('I_w, I_c, I_t')
m_w, m_c, m_t, g, t, x_tot, m_tot = symbols('m_w, m_c, m_t, g, t, x_tot, m_tot')

# related to slope
alpha = symbols('alpha')
m_d, l_d, h_d = symbols('m_d, l_d, h_d')


In [4]:
# # Physical Parameter, KITECH Robot
# param = {h_t : 0.28433, I_t : 179.5035, m_t : 424.561+200, 
#          h_c : 0.05774, I_c : 0.4868, m_c : 31.914,
#          r : 0.2206625, I_w : 0.1367, m_w:9.595, g:9.81}

# # param2 = {h_t : 0.11198, I_t : 0.214776135, m_t : 8.885+200, 
# #          h_c : 0.034617, I_c : 0.003414834, m_c : 4.124,
# #          r : 0.065, I_w : 0.004647634, m_w:2.0, g:9.81}

# param = {r:0.065, h_c:0.034617, h_t:0.11198, m_w:2, m_c:4.124, m_t:8.885, g:9.8, alpha:0*np.pi/180, x.diff().diff():0.5, x_tot:0.05, m_tot:9.385 }

# # Moment of Inertia
# param[I_w] = 0.1367 #0.2734 #0.1367
# param[I_c] = 0.4868 #0.6541
# param[I_t] = 179.5035 #173.4619

# #r : 0.2206625
# #h_t : 0.28433
# #h_c : 0.05774

# #m_t : 368.2986
# #m_c : 31.914
# #m_w : 9.595

# #I_t : 179.5035
# #I_c : 0.4868
# #I_w : 0.1367

# #g : 9.81

In [5]:
# Physical Parameter, KITECH Robot v2
param = {h_t : 0.30504, I_t : 175.3832154, m_t : 307.332, 
         h_c : 0.00203, I_c : 1.154265591, m_c : 38.72018,
         r : 0.22, I_w : 0.189170789, m_w:12.05514, g:9.81}

# param2 = {h_t : 0.11198, I_t : 0.214776135, m_t : 8.885+200, 
#          h_c : 0.034617, I_c : 0.003414834, m_c : 4.124,
#          r : 0.065, I_w : 0.004647634, m_w:2.0, g:9.81}

# param = {r:0.065, h_c:0.034617, h_t:0.11198, m_w:2, m_c:4.124, m_t:8.885, g:9.8, alpha:0*np.pi/180, x.diff().diff():0.5, x_tot:0.05, m_tot:9.385 }

# Moment of Inertia
# param[I_w] = 0.1367 #0.2734 #0.1367
# param[I_c] = 0.4868 #0.6541
# param[I_t] = 179.5035 #173.4619

#r : 0.2206625
#h_t : 0.28433
#h_c : 0.05774

#m_t : 368.2986
#m_c : 31.914
#m_w : 9.595

#I_t : 179.5035
#I_c : 0.4868
#I_w : 0.1367

#g : 9.81

In [6]:
# Newtonian Reference Frames
N = ReferenceFrame('N')
No = Point('No') 
No.set_vel(N, 0)

# Slope
S = N.orientnew('S', 'Axis', [-alpha, N.y])

# Wheel Center Point
Wo = No.locatenew('Wo', x*S.x + r*S.z)
Wo.set_vel(S, Wo.pos_from(No).diff(t, N))

# Pendulum 
P = N.orientnew('P', 'Axis', [theta, N.y])
Po = Wo.locatenew('Po', h_c*P.z)
Po.set_vel(P, 0)
J_pend = inertia(P, 0, I_c, 0)
Pend = RigidBody('Pend', Po, P, m_c, (J_pend, Po))

# Torso
T = P.orientnew('T', 'Axis', [0, P.y])
To = Wo.locatenew('To', x_l*P.x + h_t*P.z)
To.set_vel(T, 0)
J_torso = inertia(T, 0, I_t, 0)
Torso = RigidBody('Torso', To, T, m_t, (J_torso, To))

# Dummy Mass (Point Mass)
D = P.orientnew('D', 'Axis', [0, P.y])
Do = Wo.locatenew('Do', l_d*P.x + h_d*P.z)
Do.set_vel(D, 0)
Dummy = Particle('Dummy', Do, m_d)

# Wheel 
W = P.orientnew('W', 'Axis', [phi, P.y])
Wo.set_vel(W, 0)
J_wheel = inertia(W, 0, I_w, 0)
Wheel = RigidBody('Wheel', Wo, W, m_w, (J_wheel, Wo))

In [7]:
# Confirmation of Position, Velocity and Acceleration
Wo.pos_from(No).express(N).args[0][0], Wo.vel(N).express(N).args[0][0], Wo.acc(N).express(N).args[0][0]

⎛⎡-r⋅sin(α) + x⋅cos(α)⎤  ⎡cos(α)⋅ẋ⎤  ⎡cos(α)⋅ẍ⎤⎞
⎜⎢                    ⎥  ⎢        ⎥  ⎢        ⎥⎟
⎜⎢         0          ⎥, ⎢   0    ⎥, ⎢   0    ⎥⎟
⎜⎢                    ⎥  ⎢        ⎥  ⎢        ⎥⎟
⎝⎣r⋅cos(α) + x⋅sin(α) ⎦  ⎣sin(α)⋅ẋ⎦  ⎣sin(α)⋅ẍ⎦⎠

In [8]:
# Confirmation of Position, Velocity and Acceleration
Po.pos_from(No).express(N).args[0][0], Po.vel(N).express(N).args[0][0], Po.acc(N).express(N).args[0][0]

⎛                                                                ⎡            
⎜⎡h_c⋅sin(θ) - r⋅sin(α) + x⋅cos(α)⎤  ⎡h_c⋅cos(θ)⋅θ̇ + cos(α)⋅ẋ ⎤  ⎢- h_c⋅sin(
⎜⎢                                ⎥  ⎢                        ⎥  ⎢            
⎜⎢               0                ⎥, ⎢           0            ⎥, ⎢            
⎜⎢                                ⎥  ⎢                        ⎥  ⎢            
⎜⎣h_c⋅cos(θ) + r⋅cos(α) + x⋅sin(α)⎦  ⎣-h_c⋅sin(θ)⋅θ̇ + sin(α)⋅ẋ⎦  ⎢          
⎝                                                                ⎣-h_c⋅sin(θ)⋅

  2                          ⎤⎞
θ)⋅θ̇  + h_c⋅cos(θ)⋅θ̈ + cos(α)⋅ẍ⎥⎟
                             ⎥⎟
        0                    ⎥⎟
                             ⎥⎟
                  2            ⎥⎟
θ̈ - h_c⋅cos(θ)⋅θ̇  + sin(α)⋅ẍ ⎦⎠

In [9]:
# Confirmation of Position, Velocity and Acceleration
To.pos_from(No).express(N).args[0][0], To.vel(N).express(N).args[0][0], To.acc(N).express(N).args[0][0]

⎛                                                                             
⎜⎡hₜ⋅sin(θ) - r⋅sin(α) + x⋅cos(α) + xₗ⋅cos(θ)⎤  ⎡(hₜ⋅θ̇ + xₗ̇)⋅cos(θ) - xₗ⋅sin
⎜⎢                                           ⎥  ⎢                             
⎜⎢                     0                     ⎥, ⎢                     0       
⎜⎢                                           ⎥  ⎢                             
⎜⎣hₜ⋅cos(θ) + r⋅cos(α) + x⋅sin(α) - xₗ⋅sin(θ)⎦  ⎣-(hₜ⋅θ̇ + xₗ̇)⋅sin(θ) - xₗ⋅co
⎝                                                                             

                  ⎡ ⎛           2     ⎞                                       
(θ)⋅θ̇ + cos(α)⋅ẋ ⎤  ⎢ ⎝hₜ⋅θ̈ - xₗ⋅θ̇  + xₗ̈⎠⋅cos(θ) + (-(hₜ⋅θ̇ + xₗ̇)⋅θ̇ - x
               ⎥  ⎢                                                           
               ⎥, ⎢                                       0                   
               ⎥  ⎢                                                           
s(θ)⋅θ̇ + sin(α)⋅ẋ⎦  ⎢  ⎛           2     ⎞       

In [10]:
# Confirmation of dcm, angular velocity and acceleration
P.dcm(N), P.ang_vel_in(N).express(N).args[0][0], P.ang_acc_in(N).express(N).args[0][0]

⎛⎡cos(θ)  0  -sin(θ)⎤  ⎡0⎤  ⎡0⎤⎞
⎜⎢                  ⎥  ⎢ ⎥  ⎢ ⎥⎟
⎜⎢  0     1     0   ⎥, ⎢θ̇⎥, ⎢θ̈⎥⎟
⎜⎢                  ⎥  ⎢ ⎥  ⎢ ⎥⎟
⎝⎣sin(θ)  0  cos(θ) ⎦  ⎣0⎦  ⎣0⎦⎠

In [11]:
# Confirmation of dcm, angular velocity and acceleration
T.dcm(N), T.ang_vel_in(N).express(N).args[0][0], T.ang_acc_in(N).express(N).args[0][0]

⎛⎡cos(θ)  0  -sin(θ)⎤  ⎡0⎤  ⎡0⎤⎞
⎜⎢                  ⎥  ⎢ ⎥  ⎢ ⎥⎟
⎜⎢  0     1     0   ⎥, ⎢θ̇⎥, ⎢θ̈⎥⎟
⎜⎢                  ⎥  ⎢ ⎥  ⎢ ⎥⎟
⎝⎣sin(θ)  0  cos(θ) ⎦  ⎣0⎦  ⎣0⎦⎠

In [12]:
# Confirmation of dcm, angular velocity and acceleration
W.dcm(N), W.ang_vel_in(N).express(N).args[0][0], W.ang_acc_in(N).express(N).args[0][0]

⎛⎡-sin(φ)⋅sin(θ) + cos(φ)⋅cos(θ)  0  -sin(φ)⋅cos(θ) - sin(θ)⋅cos(φ)⎤  ⎡  0  ⎤ 
⎜⎢                                                                 ⎥  ⎢     ⎥ 
⎜⎢              0                 1                0               ⎥, ⎢φ̇ + θ̇
⎜⎢                                                                 ⎥  ⎢     ⎥ 
⎝⎣sin(φ)⋅cos(θ) + sin(θ)⋅cos(φ)   0  -sin(φ)⋅sin(θ) + cos(φ)⋅cos(θ)⎦  ⎣  0  ⎦ 

 ⎡  0  ⎤⎞
 ⎢     ⎥⎟
⎥, ⎢φ̈ + θ̈⎥⎟
 ⎢     ⎥⎟
 ⎣  0  ⎦⎠

In [13]:
# point where the wheels contact the ground
Wn = Wo.locatenew('Wn', -r*N.z)
Wn.v2pt_theory(Wo, N, W)

ẋ s_x + -r⋅(φ̇ + θ̇) n_x

In [14]:
# Rolling constraint
# Express the velocity of points in the inertial frame
constraints = Wn.vel(N).express(N).args[0][0]
simplify(constraints)

⎡-r⋅(φ̇ + θ̇) + cos(α)⋅ẋ⎤
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎣      sin(α)⋅ẋ       ⎦

In [15]:
con = solve(constraints, [phi.diff()])
con_rhs = Matrix(list(con.values()))
con_lhs = Matrix(list(con.keys()))

con_rhs, con_lhs, con

⎛⎡     cos(α)⋅ẋ⎤       ⎧        cos(α)⋅ẋ⎫⎞
⎜⎢-θ̇ + ────────⎥, [φ̇], ⎨φ̇: -θ̇ + ────────⎬⎟
⎝⎣        r    ⎦       ⎩           r    ⎭⎠

In [16]:
# Generalized coordinates
q = Matrix([[x], [x_l], [theta]])
#q = Matrix([[x], [theta]])
qd = q.diff()
qdd = qd.diff()

In [17]:
# Define forces on system (point, vector) tuple:
flist = [(Wo, -m_w*g*N.z), 
         (Po, -m_c*g*N.z), 
         (To, -m_t*g*N.z), 
         (Do, -m_d*g*N.z), 
         (Wo, F*N.x), 
         (To, F_l*T.x),
         (P, -F_l*h_t*P.y)] 

In [18]:
# Lagranges Method
Lag = Lagrangian(N, Pend, Torso, Dummy, Wheel)
nonslip_condition = {con_lhs[0]:con_rhs[0]}
Lag_constrainted = msubs(Lag, nonslip_condition)
Le = LagrangesMethod(Lag_constrainted, q, forcelist=flist, frame=N)
eoms = Le.form_lagranges_equations()
eoms_simple = simplify(eoms)
eoms_simple 

⎡       2                                                                     
⎢I_w⋅cos (α)⋅ẍ                                                               
⎢───────────── + g⋅m_c⋅sin(α) + g⋅m_d⋅sin(α) + g⋅mₜ⋅sin(α) + g⋅m_w⋅sin(α) - h_
⎢       2                                                                     
⎢      r                                                                      
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                 I_c⋅θ̈ + Iₜ⋅θ̈ - g⋅h_c⋅m_c⋅s

                                                                              
                   2                               

In [19]:
inv_dyn = get_Simplified_EoM(eoms_simple, q)
inv_dyn

⎡                                                                             
⎢                                                                             
⎢g⋅m_c⋅sin(α) + g⋅m_d⋅sin(α) + g⋅mₜ⋅sin(α) + g⋅m_w⋅sin(α) + mₜ⋅(-sin(α)⋅sin(θ)
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                                             

                                                                              
                                                   

In [20]:
# inv_dyn_single = inv_dyn.subs({x_l:0, F_l:0})
# inv_dyn_single

In [21]:
# linearlize_eq = {sin(theta):theta, cos(theta):1, theta.diff():0}
linearlize_eq = {alpha:theta}
inv_dyn0 = msubs(inv_dyn, linearlize_eq)
# inv_dyn_linear = msubs(inv_dyn_single, linearlize_eq)
inv_dyn0

⎡                                                                             
⎢                                                              ⎛     2        
⎢g⋅m_c⋅sin(θ) + g⋅m_d⋅sin(θ) + g⋅mₜ⋅sin(θ) + g⋅m_w⋅sin(θ) + mₜ⋅⎝- sin (θ) + co
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                                             

                                                                              
 2   ⎞                                ⎛            

In [22]:
linearlize_eq2 = {sin(2*theta):theta, cos(2*theta):1, sin(theta):theta, cos(theta):1, theta.diff():0, x_l.diff():0, x_l:0, m_d:0, l_d:0}
inv_dyn_linear = msubs(inv_dyn0, linearlize_eq2)
# inv_dyn_linear = msubs(inv_dyn_single, linearlize_eq)
inv_dyn_linear

⎡                                                                             
⎢                                ⎛     2⎞      ⎛           2                  
⎢g⋅m_c⋅θ + g⋅mₜ⋅θ + g⋅m_w⋅θ + mₜ⋅⎝1 - θ ⎠⋅xₗ̈ + ⎝- h_c⋅m_c⋅θ  + h_c⋅m_c - hₜ⋅m
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                        ⎛    
⎢                                                 -g⋅mₜ⋅θ + hₜ⋅mₜ⋅θ̈ + mₜ⋅⎝1 -
⎢                                                                             
⎢                                                   ⎛              2         2
⎣             -g⋅(h_c⋅m_c⋅θ + hₜ⋅mₜ⋅θ) + hₜ⋅mₜ⋅xₗ̈ + ⎝I_c + Iₜ + h_c ⋅m_c + hₜ

                                  ⎛           2       2        2⎞  ⎤
  2        ⎞             2        ⎝I_w + m_c⋅r  + mₜ⋅r  + m_w

In [23]:
# Control Input Variable
u = Matrix([[F], [F_l]])
#u = Matrix([[F]])

# Inverse Dynamics Equation
# M(q)*qdd + C(q,qd) + G(q) = W*u
M, C, G, W = get_EoM_from_T(inv_dyn,qdd,g,u)
#M, C, G, W = get_EoM_from_T(inv_dyn_single,qdd,g,u)
M, C, G, W

⎛⎡                                       2                                    
⎜⎢                                I_w⋅cos (α)                                 
⎜⎢                                ─────────── + m_c + m_d + mₜ + m_w          
⎜⎢                                      2                                     
⎜⎢                                     r                                      
⎜⎢                                                                            
⎜⎢                                          mₜ⋅cos(α + θ)                     
⎜⎢                                                                            
⎜⎢                                                                            
⎝⎣h_c⋅m_c⋅cos(α + θ) + h_d⋅m_d⋅cos(α + θ) + hₜ⋅mₜ⋅cos(α + θ) - l_d⋅m_d⋅sin(α +

                                                                              
                                                                              
                        mₜ⋅cos(α + θ)  h_c⋅m_c⋅cos(

In [24]:
# Dummy Mass에 의해 md 추가시 평형상태 방정식
ss_eq = {theta:0, theta.diff():0, theta.diff().diff():0, x.diff():0, x.diff().diff():0, x_l.diff():0, x_l.diff().diff():0}
inv_dyn_ss0 = msubs(inv_dyn, ss_eq)

inv_dyn_ss0

⎡g⋅m_c⋅sin(α) + g⋅m_d⋅sin(α) + g⋅mₜ⋅sin(α) + g⋅m_w⋅sin(α) - F⋅cos(α) - Fₗ⋅cos(
⎢                                                                             
⎢                                      -Fₗ                                    
⎢                                                                             
⎣                             -g⋅(l_d⋅m_d + mₜ⋅xₗ)                            

α)⎤
  ⎥
  ⎥
  ⎥
  ⎦

In [25]:
# xl 래퍼런스
ss_eq = {theta.diff():0, theta.diff().diff():0, x_l.diff():0, x_l.diff().diff():0}
xl_dy = msubs(inv_dyn, ss_eq)

solve(xl_dy[2], x_l)

⎡-g⋅h_c⋅m_c⋅sin(θ) - g⋅h_d⋅m_d⋅sin(θ) - g⋅hₜ⋅mₜ⋅sin(θ) - g⋅l_d⋅m_d⋅cos(θ) + h_
⎢─────────────────────────────────────────────────────────────────────────────
⎣                                                                  mₜ⋅(g⋅cos(θ

c⋅m_c⋅cos(α + θ)⋅ẍ + h_d⋅m_d⋅cos(α + θ)⋅ẍ + hₜ⋅mₜ⋅cos(α + θ)⋅ẍ - l_d⋅m_d⋅si
──────────────────────────────────────────────────────────────────────────────
) + sin(α + θ)⋅ẍ)                                                            

n(α + θ)⋅ẍ⎤
───────⎥
       ⎦

In [26]:
# 평지일경우
# xl_dy = msubs(inv_dyn_ss0, {alpha:0, l_d:0.5, m_d:200, m_t:424.51})
xl_dy2 = msubs(xl_dy, {alpha:0, theta:0})

solve(xl_dy2[2], x_l)


⎡-g⋅l_d⋅m_d + hₜ⋅mₜ⋅ẍ + (h_c⋅m_c + h_d⋅m_d)⋅ẍ⎤
⎢────────────────────────────────────────────⎥
⎣                    g⋅mₜ                    ⎦

In [27]:
# 평지일경우
# xl_dy = msubs(inv_dyn_ss0, {alpha:0, l_d:0.5, m_d:200, m_t:424.51})
ss_eq2 = {theta.diff():0, theta.diff().diff():0, x.diff():0, x.diff().diff():0, x_l.diff():0, x_l.diff().diff():0}
xl_dy2 = msubs(inv_dyn, ss_eq2)

solve(xl_dy2[2], x_l)


⎡-(h_c⋅m_c⋅tan(θ) + h_d⋅m_d⋅tan(θ) + hₜ⋅mₜ⋅tan(θ) + l_d⋅m_d) ⎤
⎢────────────────────────────────────────────────────────────⎥
⎣                             mₜ                             ⎦

In [28]:
# 경사로에서 theta가 경사각도 alpha와 같게 하는 xl 값
ss_eq = {theta:alpha, theta.diff():0, theta.diff().diff():0, x.diff():0, x.diff().diff():0, x_l.diff():0, x_l.diff().diff():0}
inv_dyn_ss1 = msubs(inv_dyn, ss_eq)

solve(inv_dyn_ss1[2], x_l)

⎡-(h_c⋅m_c⋅tan(α) + h_d⋅m_d⋅tan(α) + hₜ⋅mₜ⋅tan(α) + l_d⋅m_d) ⎤
⎢────────────────────────────────────────────────────────────⎥
⎣                             mₜ                             ⎦

In [41]:
# 경사로에서 theta가 경사각도 alpha와 같게 하는 xl 값
ss_eq = {theta.diff():0, theta.diff().diff():0, x.diff():0, x_l.diff():0, x_l.diff().diff():0}
inv_dyn_ss1 = msubs(inv_dyn, ss_eq)

solve(inv_dyn_ss1[2], x_l)

⎡-g⋅h_c⋅m_c⋅sin(θ) - g⋅h_d⋅m_d⋅sin(θ) - g⋅hₜ⋅mₜ⋅sin(θ) - g⋅l_d⋅m_d⋅cos(θ) + h_
⎢─────────────────────────────────────────────────────────────────────────────
⎣                                                                  mₜ⋅(g⋅cos(θ

c⋅m_c⋅cos(α + θ)⋅ẍ + h_d⋅m_d⋅cos(α + θ)⋅ẍ + hₜ⋅mₜ⋅cos(α + θ)⋅ẍ - l_d⋅m_d⋅si
──────────────────────────────────────────────────────────────────────────────
) + sin(α + θ)⋅ẍ)                                                            

n(α + θ)⋅ẍ⎤
───────⎥
       ⎦

In [30]:
solve(G[2], x_l)

⎡-(h_c⋅m_c⋅tan(θ) + h_d⋅m_d⋅tan(θ) + hₜ⋅mₜ⋅tan(θ) + l_d⋅m_d) ⎤
⎢────────────────────────────────────────────────────────────⎥
⎣                             mₜ                             ⎦

In [31]:
# Linearized Model
Ml, Cl, Gl, Wl = get_EoM_from_T(inv_dyn_linear,qdd,g,u)

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)
Mlp, Clp, Glp, Wlp

⎛⎡                                                     2                      
⎜⎢        362.015807376033          307.332 - 307.332⋅θ   93.8271552454 - 93.8
⎜⎢                                                                            
⎜⎢                         2                                                  
⎜⎢      307.332 - 307.332⋅θ               307.332                   93.7485532
⎜⎢                                                                            
⎜⎢                               2                                            
⎝⎣93.8271552454 - 93.8271552454⋅θ       93.74855328               205.13469924

           2⎤                                         ⎞
271552454⋅θ ⎥                              ⎡        2⎤⎟
            ⎥  ⎡0⎤  ⎡  3513.0328092⋅θ   ⎤  ⎢1  1 - θ ⎥⎟
            ⎥  ⎢ ⎥  ⎢                   ⎥  ⎢         ⎥⎟
8           ⎥, ⎢0⎥, ⎢   -3014.92692⋅θ   ⎥, ⎢0    1   ⎥⎟
            ⎥  ⎢ ⎥  ⎢                   ⎥  ⎢         ⎥⎟
            ⎥  ⎣0⎦  ⎣-920.44439

In [32]:
# Linearized Model
linearlize_eq1 = {alpha:theta}
linearlize_eq2 = {sin(2*theta):theta, cos(2*theta):1, sin(theta):theta, cos(theta):1, theta.diff():0, x_l.diff():0, x_l:0, m_d:0, l_d:0}

Ml =M.subs(linearlize_eq1)
Cl =C.subs(linearlize_eq1)
Gl =G.subs(linearlize_eq1)
Wl =W.subs(linearlize_eq1)
# Ml, Cl, Gl, Wl

Mlp = Ml.subs(linearlize_eq2)
Clp = Cl.subs(linearlize_eq2)
Glp = Gl.subs(linearlize_eq2)
Wlp = Wl.subs(linearlize_eq2)
Mlp, Clp, Glp, Wlp

⎛⎡I_w                                                      ⎤                  
⎜⎢─── + m_c + mₜ + m_w   mₜ          h_c⋅m_c + hₜ⋅mₜ       ⎥                  
⎜⎢  2                                                      ⎥  ⎡0⎤  ⎡  g⋅(m_c +
⎜⎢ r                                                       ⎥  ⎢ ⎥  ⎢          
⎜⎢                                                         ⎥, ⎢0⎥, ⎢        -g
⎜⎢         mₜ            mₜ               hₜ⋅mₜ            ⎥  ⎢ ⎥  ⎢          
⎜⎢                                                         ⎥  ⎣0⎦  ⎣-g⋅(h_c⋅m_
⎜⎢                                           2         2   ⎥                  
⎝⎣  h_c⋅m_c + hₜ⋅mₜ     hₜ⋅mₜ  I_c + Iₜ + h_c ⋅m_c + hₜ ⋅mₜ⎦                  

                       ⎞
                       ⎟
 mₜ + m_w)⋅θ  ⎤  ⎡1  1⎤⎟
              ⎥  ⎢    ⎥⎟
⋅mₜ⋅θ         ⎥, ⎢0  1⎥⎟
              ⎥  ⎢    ⎥⎟
c⋅θ + hₜ⋅mₜ⋅θ)⎦  ⎣0  0⎦⎟
                       ⎟
                       ⎠

In [33]:
Mlp = msubs(Mlp, param)
Clp = msubs(Clp, param)
Glp = msubs(Glp, param)
Wlp = msubs(Wlp, param)
Mlp, Clp, Glp, Wlp

⎛⎡362.015807376033    307.332     93.8271552454  ⎤  ⎡0⎤  ⎡  3513.0328092⋅θ   ⎤
⎜⎢                                               ⎥  ⎢ ⎥  ⎢                   ⎥
⎜⎢    307.332         307.332      93.74855328   ⎥, ⎢0⎥, ⎢   -3014.92692⋅θ   ⎥
⎜⎢                                               ⎥  ⎢ ⎥  ⎢                   ⎥
⎝⎣ 93.8271552454    93.74855328  205.134699245521⎦  ⎣0⎦  ⎣-920.444392957374⋅θ⎦

  ⎡1  1⎤⎞
  ⎢    ⎥⎟
, ⎢0  1⎥⎟
  ⎢    ⎥⎟
  ⎣0  0⎦⎠

In [34]:
# Forward Dynamics eqation
# qdd = M(q)^-1 * ( -C(q,qd) - G(q) + W*u )
Mlp_inv = simplify(Mlp.inv())
qdd_rhs_A = simplify(Mlp_inv*(-Clp -Glp))
qdd_rhs_B = simplify(Mlp_inv*Wlp*u)
qdd_rhs_A, qdd_rhs_B

⎛⎡-119.376549724096⋅θ⎤  ⎡ 0.0182869607653745⋅F + 2.4836729819247e-6⋅Fₗ  ⎤⎞
⎜⎢                   ⎥  ⎢                                               ⎥⎟
⎜⎢129.169004045112⋅θ ⎥, ⎢-0.0182844770923926⋅F + 0.00377840654765642⋅Fₗ ⎥⎟
⎜⎢                   ⎥  ⎢                                               ⎥⎟
⎝⎣0.057519272829003⋅θ⎦  ⎣-8.14212228535614e-6⋅F - 0.00172790456622205⋅Fₗ⎦⎠

In [35]:
# # Nonlinear Dynamics Model
# Mp = msubs(M, param)
# Cp = msubs(C, param)
# Gp = msubs(G, param)
# Wp = msubs(W, param)
# Mp, Cp, Gp, Wp

In [36]:
# # Forward Dynamics eqation
# # qdd = M(q)^-1 * ( -C(q,qd) - G(q) + W*u )
# Mp_inv = (Mp.inv())
# qdd_rhs_A_nonL = (Mp_inv*(-Cp -Gp))
# qdd_rhs_B_nonL = (Mp_inv*Wp*u)
# qdd_rhs_A_nonL,qdd_rhs_B_nonL

In [37]:
# State-space equation. 
X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u
X, U, Xd_A, Xd_B

⎛⎡x ⎤        ⎡         ẋ         ⎤  ⎡                       ẋ               
⎜⎢  ⎥        ⎢                   ⎥  ⎢                                         
⎜⎢xₗ⎥        ⎢        xₗ̇         ⎥  ⎢                      xₗ̇               
⎜⎢  ⎥        ⎢                   ⎥  ⎢                                         
⎜⎢θ ⎥  ⎡F ⎤  ⎢         θ̇         ⎥  ⎢                       θ̇               
⎜⎢  ⎥, ⎢  ⎥, ⎢                   ⎥, ⎢                                         
⎜⎢ẋ ⎥  ⎣Fₗ⎦  ⎢-119.376549724096⋅θ⎥  ⎢ 0.0182869607653745⋅F + 2.4836729819247e
⎜⎢  ⎥        ⎢                   ⎥  ⎢                                         
⎜⎢xₗ̇⎥        ⎢129.169004045112⋅θ ⎥  ⎢-0.0182844770923926⋅F + 0.00377840654765
⎜⎢  ⎥        ⎢                   ⎥  ⎢                                         
⎝⎣θ̇ ⎦        ⎣0.057519272829003⋅θ⎦  ⎣-8.14212228535614e-6⋅F - 0.0017279045662

        ⎤⎞
      ⎥⎟
        ⎥⎟
      ⎥⎟
        ⎥⎟
      ⎥⎟
-6⋅Fₗ  ⎥⎟
      ⎥⎟
642⋅Fₗ ⎥⎟
      ⎥⎟
2205⋅Fₗ⎦⎠

In [38]:
# State-space equation
A = simplify(Xd_A.jacobian(X))
B = simplify(Xd_B.jacobian(U))
C = simplify(X.jacobian(X))
D = simplify(X.jacobian(U))
A, B, C, D

⎛⎡0  0          0          1  0  0⎤  ⎡         0                     0        
⎜⎢                                ⎥  ⎢                                        
⎜⎢0  0          0          0  1  0⎥  ⎢         0                     0        
⎜⎢                                ⎥  ⎢                                        
⎜⎢0  0          0          0  0  1⎥  ⎢         0                     0        
⎜⎢                                ⎥, ⎢                                        
⎜⎢0  0  -119.376549724096  0  0  0⎥  ⎢ 0.0182869607653745    2.4836729819247e-
⎜⎢                                ⎥  ⎢                                        
⎜⎢0  0  129.169004045112   0  0  0⎥  ⎢-0.0182844770923926   0.0037784065476564
⎜⎢                                ⎥  ⎢                                        
⎝⎣0  0  0.057519272829003  0  0  0⎦  ⎣-8.14212228535614e-6  -0.001727904566222

  ⎤  ⎡1  0  0  0  0  0⎤  ⎡0  0⎤⎞
  ⎥  ⎢                ⎥  ⎢    ⎥⎟
  ⎥  ⎢0  1  0  0  0  0⎥  ⎢0  0⎥⎟
  ⎥  ⎢                ⎥  ⎢    ⎥

In [39]:
np.array(A)

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, -119.376549724096, 0, 0, 0],
       [0, 0, 129.169004045112, 0, 0, 0],
       [0, 0, 0.0575192728290030, 0, 0, 0]], dtype=object)

In [40]:
np.array(B)

array([[0, 0],
       [0, 0],
       [0, 0],
       [0.0182869607653745, 2.48367298192470e-6],
       [-0.0182844770923926, 0.00377840654765642],
       [-8.14212228535614e-6, -0.00172790456622205]], dtype=object)